
# Finance & Analytics Club

## Renko Chart

It is built using price movement and not price against standardised time intervals - this filters out the noise and lets you visualize the true trend.

Price movements are represented as bricks stacked at 45 degrees to each other. A new brick is added to the chart only when the price moves by a pre determined amount in either direction.

Renko Charts have a time axis, but the time scale is not fixed. Some bricks may take longer to form than other, depending on the time it takes the price to move the required brick size.

Renko charts typically use only closing price based on the chart time framne choosen.

More at: https://www.investopedia.com/terms/r/renkochart.asp

Import necessary libraries

In [1]:
import pandas_datareader.data as pdr
import datetime
from stocktrends import Renko #go through the documentation page and download stocktrends https://github.com/ChillarAnand/stocktrends

Download historical data for required stocks

In [2]:
ticker = "AAPL"
ohlcv = pdr.get_data_yahoo(ticker,datetime.date.today()-datetime.timedelta(364),datetime.date.today())
ohlcv

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2019-06-27,201.570007,199.570007,200.289993,199.740005,20899700.0,197.384323
2019-06-28,199.500000,197.050003,198.679993,197.919998,31110600.0,195.585770
2019-07-01,204.490005,200.649994,203.169998,201.550003,27316700.0,199.172989
2019-07-02,203.130005,201.360001,201.410004,202.729996,16935200.0,200.339050
2019-07-03,204.440002,202.690002,203.279999,204.410004,11362000.0,201.999237
...,...,...,...,...,...,...
2020-06-19,356.559998,345.149994,354.640015,349.720001,66119000.0,349.720001
2020-06-22,359.459991,351.149994,351.339996,358.869995,33861300.0,358.869995
2020-06-23,372.380005,362.269989,364.000000,366.529999,53038900.0,366.529999


### Function to calculate True Range and Average True Range

In [3]:
def ATR(DF,n):
    df = DF.copy()
    df['H-L']=abs(df['High']-df['Low'])
    df['H-PC']=abs(df['High']-df['Adj Close'].shift(1))
    df['L-PC']=abs(df['Low']-df['Adj Close'].shift(1))
    df['TR']=df[['H-L','H-PC','L-PC']].max(axis=1,skipna=False)
    df['ATR'] = df['TR'].rolling(n).mean()
    df2 = df.drop(['H-L','H-PC','L-PC'],axis=1)
    return df2

In [4]:
df2 = ATR(ohlcv, 14)
df2 = df2.dropna()
df2

,High,Low,Open,Close,Volume,Adj Close,TR,ATR
Date,,,,,,,,
2019-07-18,205.880005,203.699997,204.000000,205.660004,18582200.0,203.234497,4.928268,4.215771
2019-07-19,206.500000,202.360001,205.789993,202.589996,20929300.0,200.200699,4.139999,4.336486
2019-07-22,207.229996,203.610001,203.649994,207.220001,22277900.0,204.776108,7.029297,4.202562
2019-07-23,208.910004,207.289993,208.460007,208.839996,18355200.0,206.376984,4.133896,4.215196
2019-07-24,209.149994,207.169998,207.669998,208.669998,14991600.0,206.209015,2.773010,4.120343
...,...,...,...,...,...,...,...,...
2020-06-19,356.559998,345.149994,354.640015,349.720001,66119000.0,349.720001,11.410004,9.110001
2020-06-22,359.459991,351.149994,351.339996,358.869995,33861300.0,358.869995,9.739990,9.483571
2020-06-23,372.380005,362.269989,364.000000,366.529999,53038900.0,366.529999,13.510010,10.169998


### Function to convert ohlc data into renko bricks

In [5]:
def renko_DF(DF):
    df = DF.copy()
    df.reset_index(inplace=True)
    df = df.iloc[:,[0,1,2,3,5,6]]
    df.rename(columns = {"Date" : "date", "High" : "high","Low" : "low", "Open" : "open","Adj Close" : "close", "Volume" : "volume"}, 
              inplace = True) # we have to change the column names as mentioned in the documentation in order to make it work!
    df2 = Renko(df)
    df2.brick_size = round(ATR(DF,120)["ATR"][-1],0)
    renko_df = df2.get_ohlc_data() 
    return renko_df

In [6]:
df = renko_DF(df2)
df = df.dropna()
df

,date,open,high,low,close,uptrend
0,2019-07-18,190.0,200.0,190.0,200.0,True
1,2019-07-31,200.0,210.0,200.0,210.0,True
2,2019-09-11,210.0,220.0,210.0,220.0,True
3,2019-10-11,220.0,230.0,220.0,230.0,True
4,2019-10-23,230.0,240.0,230.0,240.0,True
5,2019-11-01,240.0,250.0,240.0,250.0,True
6,2019-11-11,250.0,260.0,250.0,260.0,True
7,2019-12-12,260.0,270.0,260.0,270.0,True
8,2019-12-23,270.0,280.0,270.0,280.0,True
9,2019-12-30,280.0,290.0,280.0,290.0,True
